# Groq bot
Let's connect everything together...

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH__PROJECT")

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "gemma2-9b-it")
llm

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory # store and manage the conversation's messages in a structured way..
from langchain_core.chat_history import BaseChatMessageHistory # adding, retrieving, or clearing messages) for any chat history implementation..
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [ ]:
store

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistence, provide answers to the query in short and answer should be as accurate as possible."),
    MessagesPlaceholder(variable_name="messages")
])

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [ ]:
# config = {"session_id": "d9"} wrong..
config = {"configurable": {"session_id": "d9"}}

In [172]:
messages = store[config["session_id"]].messages

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")| trimmer)| prompt | llm
)

In [175]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [ ]:
from langchain_core.messages import HumanMessage
with_message_history.invoke({"messages" : [HumanMessage(content="Tell me about me?")]}, config=config)

AIMessage(content='Since I have no memory of past conversations, I don\'t know anything about you.  \n\nTo tell you something about yourself, I need you to tell me something! For example, you could say:\n\n* "I like to play the guitar."\n* "I\'m learning to code."\n* "My favorite color is blue." \n\n\nLet\'s get to know each other! 😊 \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 89, 'total_tokens': 177, 'completion_time': 0.16, 'prompt_time': 0.004105054, 'queue_time': 0.09286995, 'total_time': 0.164105054}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d53b328d-1316-4d82-9c0f-40a689a02708-0', usage_metadata={'input_tokens': 89, 'output_tokens': 88, 'total_tokens': 177})

In [179]:
store

{'d9': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am a darshan, a data scientist.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Darshan! It's nice to meet you.  \n\nAs a large language model, I'm always interested in learning about people who work with data. What kind of data science work do you do?  \n\nAre you working on any interesting projects right now?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002111055, 'queue_time': 0.108818395, 'total_time': 0.109383782}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fd6b43a0-44a3-4921-a662-9a8777dd04b3-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80}), HumanMessage(content='I am working on Agentic AI.', additional_kwargs={}, response_metadata={}), AIMessage(c